# Rice Plant Stress Detection using Deep Learning
## CNN-based Multi-class Classification System

This notebook implements a Convolutional Neural Network for detecting different types of stress in rice plants including:
- Bacterial Blight
- Blast Disease
- Brown Spot
- Tungro Virus

**Author:** kishore
**Date:** October 30, 2025

## 1. Environment Setup and Dependencies

In [ ]:
# Import required libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, ResNet50V2, MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import os
import json
from pathlib import Path
import cv2
from datetime import datetime

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Configure GPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    # Use GPU 1 as requested
    tf.config.set_visible_devices(physical_devices[0], 'GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print(f"✓ GPU Enabled: {tf.test.gpu_device_name()}")
    print(f"✓ GPU Details: {physical_devices}")
else:
    print("⚠ No GPU detected, using CPU")

print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")

## 2. Data Configuration and Paths

In [ ]:
# Dataset configuration
BASE_DIR = Path(r'c:\Users\gyuva\Downloads\cap')
DATASET_DIR = BASE_DIR / 'Rice Leaf Disease Images'
MODEL_DIR = BASE_DIR / 'models'
RESULTS_DIR = BASE_DIR / 'results'

# Create directories
MODEL_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(exist_ok=True)

# Model hyperparameters
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001

# Class names
CLASS_NAMES = ['Bacterialblight', 'Blast', 'Brownspot', 'Tungro']
NUM_CLASSES = len(CLASS_NAMES)

print(f"Dataset Directory: {DATASET_DIR}")
print(f"Model Directory: {MODEL_DIR}")
print(f"Number of Classes: {NUM_CLASSES}")
print(f"Classes: {CLASS_NAMES}")

## 3. Data Loading and Preprocessing

In [ ]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 80% train, 20% validation
)

# No augmentation for validation/test data
test_datagen = ImageDataGenerator(
    rescale=1./255
)

# Load training data
train_generator = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=42
)

print(f"\nTraining samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")
print(f"Class indices: {train_generator.class_indices}")

## 4. Visualize Sample Images

In [ ]:
# Visualize sample images from each class
def plot_sample_images():
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    fig.suptitle('Sample Images from Each Disease Class', fontsize=16, fontweight='bold')
    
    for i, disease in enumerate(CLASS_NAMES):
        disease_dir = DATASET_DIR / disease
        images = list(disease_dir.glob('*.jpg'))[:2]
        
        for j, img_path in enumerate(images):
            img = cv2.imread(str(img_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            axes[j, i].imshow(img)
            axes[j, i].set_title(disease, fontweight='bold')
            axes[j, i].axis('off')
    
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / 'sample_images.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_sample_images()

## 5. Build Custom CNN Model

In [ ]:
def create_custom_cnn():
    """
    Custom CNN architecture optimized for rice disease detection
    """
    model = models.Sequential([
        # Input layer
        layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        
        # Block 1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 3
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 4
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.4),
        
        # Fully connected layers
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        
        # Output layer
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    return model

# Create model
model = create_custom_cnn()
model.summary()

## 6. Transfer Learning Model (Alternative Approach)

In [ ]:
def create_transfer_learning_model(base_model_name='EfficientNetB0'):
    """
    Create transfer learning model using pre-trained networks
    """
    # Load base model
    if base_model_name == 'EfficientNetB0':
        base_model = EfficientNetB0(
            include_top=False,
            weights='imagenet',
            input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
        )
    elif base_model_name == 'ResNet50V2':
        base_model = ResNet50V2(
            include_top=False,
            weights='imagenet',
            input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
        )
    else:
        base_model = MobileNetV2(
            include_top=False,
            weights='imagenet',
            input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
        )
    
    # Freeze base model
    base_model.trainable = False
    
    # Build model
    inputs = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    model = keras.Model(inputs, outputs)
    return model

# Uncomment to use transfer learning instead
# model = create_transfer_learning_model('EfficientNetB0')
# model.summary()

## 7. Compile Model

In [ ]:
# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy', 
             keras.metrics.Precision(name='precision'),
             keras.metrics.Recall(name='recall'),
             keras.metrics.AUC(name='auc')]
)

print("✓ Model compiled successfully")

## 8. Setup Callbacks

In [ ]:
# Define callbacks
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

callbacks = [
    # Save best model
    ModelCheckpoint(
        filepath=str(MODEL_DIR / f'rice_disease_model_best_{timestamp}.h5'),
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    
    # Early stopping
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Reduce learning rate
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    
    # TensorBoard
    TensorBoard(
        log_dir=str(RESULTS_DIR / f'logs_{timestamp}'),
        histogram_freq=1
    )
]

print("✓ Callbacks configured")

## 9. Train Model

In [ ]:
# Train the model
print("\n" + "="*70)
print("TRAINING STARTED")
print("="*70 + "\n")

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*70)
print("TRAINING COMPLETED")
print("="*70 + "\n")

## 10. Save Final Model

In [ ]:
# Save final model in multiple formats
final_model_path = MODEL_DIR / 'rice_disease_model_final.h5'
model.save(final_model_path)
print(f"✓ Model saved: {final_model_path}")

# Save in TensorFlow SavedModel format
saved_model_path = MODEL_DIR / 'rice_disease_saved_model'
model.save(saved_model_path)
print(f"✓ SavedModel saved: {saved_model_path}")

# Save model architecture as JSON
model_json = model.to_json()
with open(MODEL_DIR / 'model_architecture.json', 'w') as json_file:
    json_file.write(model_json)
print(f"✓ Model architecture saved")

# Save class names
with open(MODEL_DIR / 'class_names.json', 'w') as f:
    json.dump(CLASS_NAMES, f)
print(f"✓ Class names saved")

## 11. Visualize Training History

In [ ]:
def plot_training_history(history):
    """
    Plot training and validation metrics
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Accuracy
    axes[0, 0].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    axes[0, 0].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
    axes[0, 0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Loss
    axes[0, 1].plot(history.history['loss'], label='Train Loss', linewidth=2)
    axes[0, 1].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
    axes[0, 1].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Precision
    axes[1, 0].plot(history.history['precision'], label='Train Precision', linewidth=2)
    axes[1, 0].plot(history.history['val_precision'], label='Val Precision', linewidth=2)
    axes[1, 0].set_title('Model Precision', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Recall
    axes[1, 1].plot(history.history['recall'], label='Train Recall', linewidth=2)
    axes[1, 1].plot(history.history['val_recall'], label='Val Recall', linewidth=2)
    axes[1, 1].set_title('Model Recall', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(RESULTS_DIR / 'training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

plot_training_history(history)

## 12. Evaluate Model on Validation Set

In [ ]:
# Evaluate model
print("Evaluating model on validation set...")
val_loss, val_accuracy, val_precision, val_recall, val_auc = model.evaluate(validation_generator, verbose=1)

# Calculate F1 Score
val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall)

print("\n" + "="*70)
print("VALIDATION RESULTS")
print("="*70)
print(f"Accuracy:  {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")
print(f"Precision: {val_precision:.4f}")
print(f"Recall:    {val_recall:.4f}")
print(f"F1-Score:  {val_f1:.4f}")
print(f"AUC:       {val_auc:.4f}")
print(f"Loss:      {val_loss:.4f}")
print("="*70 + "\n")

## 13. Generate Predictions and Confusion Matrix

In [ ]:
# Get predictions
validation_generator.reset()
y_pred_probs = model.predict(validation_generator, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = validation_generator.classes

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Normalized confusion matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='RdYlGn',
            xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES,
            cbar_kws={'label': 'Percentage'})
plt.title('Normalized Confusion Matrix', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'confusion_matrix_normalized.png', dpi=300, bbox_inches='tight')
plt.show()

## 14. Classification Report

In [ ]:
# Generate detailed classification report
report = classification_report(y_true, y_pred, target_names=CLASS_NAMES, digits=4)
print("\n" + "="*70)
print("CLASSIFICATION REPORT")
print("="*70)
print(report)

# Save report to file
with open(RESULTS_DIR / 'classification_report.txt', 'w') as f:
    f.write(report)

# Get detailed metrics per class
report_dict = classification_report(y_true, y_pred, target_names=CLASS_NAMES, output_dict=True)
df_report = pd.DataFrame(report_dict).transpose()
df_report.to_csv(RESULTS_DIR / 'classification_report.csv')
print("\n✓ Classification report saved")

## 15. Save Training Metrics and Results

In [ ]:
# Save comprehensive results
results = {
    'model_architecture': 'Custom CNN',
    'timestamp': timestamp,
    'dataset': {
        'total_samples': train_generator.samples + validation_generator.samples,
        'training_samples': train_generator.samples,
        'validation_samples': validation_generator.samples,
        'num_classes': NUM_CLASSES,
        'class_names': CLASS_NAMES
    },
    'hyperparameters': {
        'image_size': f'{IMG_HEIGHT}x{IMG_WIDTH}',
        'batch_size': BATCH_SIZE,
        'epochs_trained': len(history.history['loss']),
        'initial_learning_rate': LEARNING_RATE
    },
    'final_metrics': {
        'validation_accuracy': float(val_accuracy),
        'validation_precision': float(val_precision),
        'validation_recall': float(val_recall),
        'validation_f1_score': float(val_f1),
        'validation_auc': float(val_auc),
        'validation_loss': float(val_loss)
    },
    'per_class_metrics': report_dict,
    'training_history': {
        'train_accuracy': [float(x) for x in history.history['accuracy']],
        'val_accuracy': [float(x) for x in history.history['val_accuracy']],
        'train_loss': [float(x) for x in history.history['loss']],
        'val_loss': [float(x) for x in history.history['val_loss']]
    }
}

# Save results as JSON
with open(RESULTS_DIR / 'training_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("✓ All results saved successfully")
print(f"\nResults directory: {RESULTS_DIR}")
print(f"Model directory: {MODEL_DIR}")

## 16. Test Single Image Prediction

In [ ]:
def predict_single_image(image_path, model):
    """
    Predict disease for a single image
    """
    # Load and preprocess image
    img = cv2.imread(str(image_path))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (IMG_HEIGHT, IMG_WIDTH))
    img_array = np.expand_dims(img_resized / 255.0, axis=0)
    
    # Predict
    predictions = model.predict(img_array, verbose=0)
    predicted_class_idx = np.argmax(predictions[0])
    confidence = predictions[0][predicted_class_idx]
    predicted_class = CLASS_NAMES[predicted_class_idx]
    
    # Get all probabilities
    all_probs = {CLASS_NAMES[i]: float(predictions[0][i]) for i in range(NUM_CLASSES)}
    
    return predicted_class, confidence, all_probs, img_rgb

# Test with sample images
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
for i, disease in enumerate(CLASS_NAMES):
    sample_img = list((DATASET_DIR / disease).glob('*.jpg'))[0]
    pred_class, conf, probs, img = predict_single_image(sample_img, model)
    
    axes[i].imshow(img)
    axes[i].set_title(f'True: {disease}\nPredicted: {pred_class}\nConfidence: {conf:.2%}', 
                     fontweight='bold', fontsize=10)
    axes[i].axis('off')

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'sample_predictions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Prediction function tested successfully")

## 17. Create Treatment Recommendations Database

In [ ]:
# Treatment recommendations for each disease
treatment_recommendations = {
    "Bacterialblight": {
        "disease_name": "Bacterial Blight",
        "description": "Bacterial blight is caused by Xanthomonas oryzae pv. oryzae, resulting in wilting and drying of leaves.",
        "symptoms": [
            "Water-soaked lesions on leaf tips and margins",
            "Yellow to white lesions with wavy edges",
            "Wilting of seedlings (kresek phase)",
            "Systemic infection causing plant death"
        ],
        "precautions": [
            "Use disease-free seeds",
            "Avoid excessive nitrogen fertilization",
            "Maintain proper water management",
            "Remove and destroy infected plants",
            "Practice crop rotation"
        ],
        "treatment": [
            "Apply copper-based bactericides",
            "Use resistant varieties (e.g., IR64, IR72)",
            "Spray streptocycline (200-300 ppm) at 7-10 day intervals",
            "Apply zinc sulfate (0.5%) to strengthen plants"
        ],
        "fertilizer": "Balanced NPK (10-26-26) with emphasis on potassium",
        "water_management": "Avoid over-irrigation; maintain 2-5 cm water depth",
        "severity": "High"
    },
    "Blast": {
        "disease_name": "Rice Blast",
        "description": "Rice blast is caused by fungus Magnaporthe oryzae, one of the most destructive rice diseases worldwide.",
        "symptoms": [
            "Diamond-shaped lesions with gray centers and brown margins",
            "Leaf blast: spots on leaves",
            "Neck blast: infection at neck node",
            "Panicle blast: incomplete grain filling"
        ],
        "precautions": [
            "Use certified disease-free seeds",
            "Avoid excessive nitrogen application",
            "Ensure proper spacing between plants",
            "Remove infected stubble and debris",
            "Use silicon amendments to strengthen plants"
        ],
        "treatment": [
            "Apply Tricyclazole 75% WP @ 0.6 g/liter",
            "Spray Carbendazim 50% WP @ 1 g/liter",
            "Use Azoxystrobin 25% SC @ 1 ml/liter",
            "Apply organic fungicides like neem oil (3-5%)"
        ],
        "fertilizer": "Split application of nitrogen; use potassium silicate",
        "water_management": "Intermittent irrigation to reduce humidity",
        "severity": "Very High"
    },
    "Brownspot": {
        "disease_name": "Brown Spot",
        "description": "Brown spot is caused by fungus Bipolaris oryzae, often associated with nutrient deficiency.",
        "symptoms": [
            "Circular to oval brown spots on leaves",
            "Spots with gray or whitish centers",
            "Numerous spots causing leaf withering",
            "Affects grain quality and yield"
        ],
        "precautions": [
            "Treat seeds with fungicides before sowing",
            "Ensure adequate soil nutrition",
            "Avoid water stress conditions",
            "Maintain proper plant spacing",
            "Remove infected plant debris"
        ],
        "treatment": [
            "Spray Mancozeb 75% WP @ 2.5 g/liter",
            "Apply Propiconazole 25% EC @ 1 ml/liter",
            "Use Copper oxychloride 50% WP @ 3 g/liter",
            "Seed treatment with Carbendazim @ 2 g/kg seed"
        ],
        "fertilizer": "Apply NPK (20-10-10) with micronutrients (Zinc, Iron)",
        "water_management": "Maintain consistent moisture; avoid drought stress",
        "severity": "Moderate"
    },
    "Tungro": {
        "disease_name": "Tungro Virus",
        "description": "Tungro is a viral disease transmitted by green leafhoppers, causing severe yield loss.",
        "symptoms": [
            "Yellow or orange-yellow discoloration of leaves",
            "Stunted plant growth",
            "Reduced tillering",
            "Delayed flowering and incomplete panicle formation"
        ],
        "precautions": [
            "Control vector (green leafhopper) population",
            "Use resistant varieties",
            "Remove infected plants immediately",
            "Avoid staggered planting",
            "Synchronize planting dates in community"
        ],
        "treatment": [
            "No direct cure; focus on vector control",
            "Spray Imidacloprid 17.8% SL @ 0.3 ml/liter for leafhopper",
            "Apply Thiamethoxam 25% WG @ 0.2 g/liter",
            "Use light traps to monitor and control vectors",
            "Remove and destroy infected plants"
        ],
        "fertilizer": "Moderate nitrogen; increase potassium for plant vigor",
        "water_management": "Maintain shallow water depth (2-3 cm)",
        "severity": "Very High"
    }
}

# Save recommendations
with open(MODEL_DIR / 'treatment_recommendations.json', 'w') as f:
    json.dump(treatment_recommendations, f, indent=2)

print("✓ Treatment recommendations database created")

## 18. Final Summary

In [ ]:
print("\n" + "="*70)
print("PROJECT SUMMARY")
print("="*70)
print(f"Project: Rice Plant Stress Detection System")
print(f"Model Type: Custom CNN")
print(f"Total Parameters: {model.count_params():,}")
print(f"\nDataset Statistics:")
print(f"  - Total Images: {train_generator.samples + validation_generator.samples:,}")
print(f"  - Training: {train_generator.samples:,} images")
print(f"  - Validation: {validation_generator.samples:,} images")
print(f"  - Classes: {NUM_CLASSES}")
print(f"\nFinal Model Performance:")
print(f"  - Accuracy: {val_accuracy*100:.2f}%")
print(f"  - Precision: {val_precision:.4f}")
print(f"  - Recall: {val_recall:.4f}")
print(f"  - F1-Score: {val_f1:.4f}")
print(f"\nFiles Generated:")
print(f"  - Model: rice_disease_model_final.h5")
print(f"  - SavedModel: rice_disease_saved_model/")
print(f"  - Treatment DB: treatment_recommendations.json")
print(f"  - Results: training_results.json")
print(f"  - Visualizations: *.png files")
print("="*70 + "\n")

print("✓ Training pipeline completed successfully!")
print("✓ Model is ready for deployment")
print("\nNext steps:")
print("1. Deploy model to web application")
print("2. Test with real-world images")
print("3. Monitor and retrain as needed")